In [31]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

CPU times: user 86 ms, sys: 13.8 ms, total: 99.8 ms
Wall time: 227 ms


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
import gensim
import gensim.downloader as api
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

# Download GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

# Convert GloVe format to Word2Vec format
glove_input_file = 'glove.6B.300d.txt'  # Make sure the file name matches the downloaded file
word2vec_output_file = 'word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

# Load the converted Word2Vec model
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

# Test the model
#print(model.most_similar('king'))


--2025-01-27 17:41:43--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-01-27 17:41:43--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-01-27 17:41:43--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [33]:
import pandas as pd

train_df = pd.read_csv('/content/Fake_train.csv', encoding='utf-8')
test_df = pd.read_csv('/content/Fake_test_without_labels.csv', encoding='utf-8')
df2 = pd.read_csv('/content/Fake_dev.csv', encoding='utf-8')
import pandas as pd

# Load the dataset from the CSV file
df = pd.read_csv('/content/Fake_train.csv')

# Displaying DataFrame in Colab
df

# Preprocess data
text_column = 'text'
label_column = 'label'
train_df[text_column] = train_df[text_column].fillna('')
test_df[text_column] = test_df[text_column].fillna('')


In [34]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text
max_vocab_size = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df[text_column])

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_df[text_column])
X_test = tokenizer.texts_to_sequences(test_df[text_column])
X_dev = tokenizer.texts_to_sequences(df2[text_column])

# Pad the sequences
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')
X_dev = pad_sequences(X_dev, maxlen=max_sequence_length, padding='post', truncating='post')

# Print a sample
print(X_train[:5])


[[  38  328  111  329  991    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [4551 1629    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0

In [35]:
import numpy as np

# Create the embedding matrix
embedding_dim = 300  # This should match the dimension of the GloVe file used
num_words = min(max_vocab_size, len(tokenizer.word_index) + 1)

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < num_words:
        embedding_vector = model[word] if word in model else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

print(f"Embedding matrix shape: {embedding_matrix.shape}")


Embedding matrix shape: (10000, 300)


In [36]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df[label_column])
y_dev = label_encoder.transform(df2[label_column])

# Check the encoded labels
print(y_train[:5])


[0 0 0 0 1]


In [ ]:
'''
#lstm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=num_words,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_sequence_length,
              trainable=False),  # Freeze the GloVe embeddings
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=10,
    batch_size=32,
    verbose=1
)


#lstm
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Predict on the test data
test_predictions = model.predict(X_test)

# Convert probabilities to binary labels
test_labels = (test_predictions > 0.5).astype(int).flatten()
test_df['predicted_label'] = test_labels

# Save predictions
test_df.to_csv('/content/test_predictions.csv', index=False)
print("Predictions saved to /content/test_predictions.csv")
#lstm
# Evaluate the model on the dev set
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

from sklearn.metrics import classification_report

y_dev_pred = model.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)
print(classification_report(y_dev, y_dev_labels))


import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the test dataset
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')

# Assuming the text column in the test dataset is named 'text'
test_texts = test_data['text'].values

# Tokenize and pad the test texts (ensure you use the same tokenizer as training)
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Predict labels for the test set
test_predictions = model.predict(test_padded)

# Convert predictions to binary labels (threshold = 0.5)
test_labels = (test_predictions > 0.5).astype(int).flatten()

# Add predictions to the test dataframe
test_data['predicted_label'] = test_labels

# Save the results to a new CSV file with the model name in the filename
output_path = '/content/lstm_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")


import pandas as pd

# Load the predictions dataset
predictions_data = pd.read_csv('/content/lstm_predicted_test.csv')

# Map numeric predictions (0 and 1) to descriptive labels
label_mapping = {0: 'Fake', 1: 'original'}
predictions_data['predicted_label'] = predictions_data['predicted_label'].map(label_mapping)

# Keep only the 'id' and 'predicted_label' columns
output_data = predictions_data[['Id', 'predicted_label']]

# Save the results to a new CSV file
output_path = '/content/lstm_cleaned_test.csv'
output_data.to_csv(output_path, index=False)

print(f"Cleaned predictions saved to {output_path}")

#lstm
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# Load the files
predicted_file = pd.read_csv('/content/lstm_cleaned_test.csv')
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')

# Verify column names
print("Predicted file columns:", predicted_file.columns)
print("Actual file columns:", actual_file.columns)

# Create the 'Id' column in the actual file
# Assuming the row order corresponds to IDs like Fake_01, Fake_02
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Verify the 'Id' column
print("Actual file with 'Id' column:")
print(actual_file.head())

# Merge the files on the 'Id' column
merged = pd.merge(predicted_file, actual_file, on='Id')

# Extract the true labels and predicted labels
y_true = merged['label']  # Actual labels
y_pred = merged['predicted_label']  # Predicted labels

# Compare predictions and compute metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Save the merged file with comparison for review (optional)
merged.to_csv('/content/lstm_merged_comparison.csv', index=False)
print("Merged comparison file saved to /content/lstm_merged_comparison.csv")




Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,000,000 (11.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 3,000,000 (11.44 MB)

Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 30s 193ms/step - accuracy: 0.4913 - loss: 0.6926 - val_accuracy: 0.5252 - val_loss: 0.6890
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 17s 164ms/step - accuracy: 0.5432 - loss: 0.6909 - val_accuracy: 0.5104 - val_loss: 0.6912
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 18s 138ms/step - accuracy: 0.5854 - loss: 0.6810 - val_accuracy: 0.6025 - val_loss: 0.6700
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 26s 191ms/step - accuracy: 0.6127 - loss: 0.6693 - val_accuracy: 0.5067 - val_loss: 0.6983
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 16s 150ms/step - accuracy: 0.5091 - loss: 0.6950 - val_accuracy: 0.5067 - val_loss: 0.6921
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 20s 144ms/step - accuracy: 0.5108 - loss: 0.6913 - val_accuracy: 0.5067 - val_loss: 0.6914
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 26s 197ms/step - accuracy: 0.5023 - loss: 0.6924 - val_accuracy: 0.5423 - val_loss: 0.6838
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 15s 145ms/step - accuracy: 0.5844 - loss: 0

In [45]:
#lstm+bilstm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

# Build the model with LSTM + BiLSTM
model = Sequential([
    Embedding(input_dim=num_words,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_sequence_length,
              trainable=False),  # Freeze the GloVe embeddings
    LSTM(64, return_sequences=True),  # First LSTM layer with return_sequences=True for BiLSTM
    Bidirectional(LSTM(64, return_sequences=False)),  # BiLSTM layer
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary to see the architecture
model.summary()

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=10,
    batch_size=32,
    verbose=1
)

# Evaluate the model on the dev set
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Classification report
from sklearn.metrics import classification_report

y_dev_pred = model.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)
print(classification_report(y_dev, y_dev_labels))


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ ?                           │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_14 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_7 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,000,000 (11.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 3,000,000 (11.44 MB)

Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 41s 327ms/step - accuracy: 0.5833 - loss: 0.6679 - val_accuracy: 0.6368 - val_loss: 0.6201
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 41s 330ms/step - accuracy: 0.6499 - loss: 0.6139 - val_accuracy: 0.6515 - val_loss: 0.6028
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 41s 328ms/step - accuracy: 0.6710 - loss: 0.5893 - val_accuracy: 0.6528 - val_loss: 0.6030
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 32s 308ms/step - accuracy: 0.6702 - loss: 0.5637 - val_accuracy: 0.6466 - val_loss: 0.6044
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 42s 322ms/step - accuracy: 0.6768 - loss: 0.5523 - val_accuracy: 0.6331 - val_loss: 0.6504
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 41s 325ms/step - accuracy: 0.6849 - loss: 0.5519 - val_accuracy: 0.6393 - val_loss: 0.6112
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 43s 420ms/step - accuracy: 0.6945 - loss: 0.5276 - val_accuracy: 0.6405 - val_loss: 0.6243
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 72s 321ms/step - accuracy: 0.6969 - loss: 0

In [46]:
# 1. Make Predictions on the Dev Set
y_dev_pred = model.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)

# 2. Classification Report on Dev Set
from sklearn.metrics import classification_report
print("Classification Report on Dev Set:")
print(classification_report(y_dev, y_dev_labels))

# 3. Make Predictions on the Test Set
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the test dataset
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')  # Replace with actual test data path

# Assuming the text column in the test dataset is named 'text'
test_texts = test_data['text'].values

# Tokenize and pad the test texts (ensure you use the same tokenizer as during training)
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Predict labels for the test set
test_predictions = model.predict(test_padded)

# Convert predictions to binary labels (threshold = 0.5)
test_labels = (test_predictions > 0.5).astype(int).flatten()

# 4. Add Predictions to the Test Dataframe
test_data['predicted_label'] = test_labels

# Save the results to a new CSV file with the model name in the filename
output_path = '/content/lstm_bilstm_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")

# 5. Map numeric predictions (0 and 1) to descriptive labels (Fake/Original)
label_mapping = {0: 'Fake', 1: 'original'}
test_data['predicted_label'] = test_data['predicted_label'].map(label_mapping)

# Keep only the 'Id' and 'predicted_label' columns
output_data = test_data[['Id', 'predicted_label']]

# Save the cleaned predictions to a new CSV file
output_path_cleaned = '/content/lstm_bilstm_cleaned_test.csv'
output_data.to_csv(output_path_cleaned, index=False)

print(f"Cleaned predictions saved to {output_path_cleaned}")

# 6. Evaluate the Model on the Test Set
# Assuming you have the actual labels for the test set in a separate file
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')  # Replace with actual test labels file

# Verify column names and create 'Id' column in the actual file
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Merge the predicted and actual labels on the 'Id' column
merged = pd.merge(output_data, actual_file, on='Id')

# Extract the true labels and predicted labels
y_true = merged['label']  # Actual labels
y_pred = merged['predicted_label']  # Predicted labels

# Compare predictions and compute metrics
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy on Test Set:", accuracy_score(y_true, y_pred))
print("Classification Report on Test Set:")
print(classification_report(y_true, y_pred))

# Save the merged file with comparison for review (optional)
merged.to_csv('/content/lstm_bilstm_merged_comparison.csv', index=False)
print("Merged comparison file saved to /content/lstm_bilstm_merged_comparison.csv")


26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 131ms/step
Classification Report on Dev Set:
              precision    recall  f1-score   support

           0       0.80      0.41      0.54       406
           1       0.60      0.90      0.72       409

    accuracy                           0.66       815
   macro avg       0.70      0.65      0.63       815
weighted avg       0.70      0.66      0.63       815

32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step
Predictions saved to /content/lstm_bilstm_predicted_test.csv
Cleaned predictions saved to /content/lstm_bilstm_cleaned_test.csv
Accuracy on Test Set: 0.6545632973503435
Classification Report on Test Set:
              precision    recall  f1-score   support

        Fake       0.80      0.40      0.54       507
    original       0.60      0.90      0.72       512

    accuracy                           0.65      1019
   macro avg       0.70      0.65      0.63      1019
weighted avg       0.70      0.65      0.63      1019

Merged comparison file sav

In [43]:
#bilstm
# Import necessary modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2

# Define the BiLSTM model
model_bilstm = Sequential([
    # Embedding layer with pre-trained GloVe embeddings
    Embedding(input_dim=num_words,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_sequence_length,
              trainable=False),  # Freeze the GloVe embeddings

    # BiLSTM layer - Bidirectional LSTM
    Bidirectional(LSTM(64, return_sequences=False)),  # Set return_sequences=False for classification tasks

    # Dropout layer to prevent overfitting
    Dropout(0.5),

    # Fully connected dense layer with L2 regularization
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),

    # Output layer for binary classification (sigmoid activation)
    Dense(1, activation='sigmoid')
])

# Compile the model
model_bilstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary to see the layers and parameters
model_bilstm.summary()

# Train the model
history_bilstm = model_bilstm.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=10,
    batch_size=32,
    verbose=1
)

# Evaluate the model on the dev set
val_loss, val_accuracy = model_bilstm.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Classification report for the dev set
from sklearn.metrics import classification_report
y_dev_pred = model_bilstm.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)
print("Classification Report on Dev Set:")
print(classification_report(y_dev, y_dev_labels))


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ ?                           │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,000,000 (11.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 3,000,000 (11.44 MB)

Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 44s 347ms/step - accuracy: 0.5782 - loss: 1.5091 - val_accuracy: 0.6282 - val_loss: 0.7887
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 50s 435ms/step - accuracy: 0.6380 - loss: 0.7417 - val_accuracy: 0.6344 - val_loss: 0.6408
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 35s 339ms/step - accuracy: 0.6663 - loss: 0.6198 - val_accuracy: 0.4994 - val_loss: 1.3296
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 41s 338ms/step - accuracy: 0.6524 - loss: 0.6415 - val_accuracy: 0.6393 - val_loss: 0.6176
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 40s 334ms/step - accuracy: 0.6777 - loss: 0.5848 - val_accuracy: 0.6405 - val_loss: 0.6190
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 42s 342ms/step - accuracy: 0.7003 - loss: 0.5623 - val_accuracy: 0.6356 - val_loss: 0.6342
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 41s 339ms/step - accuracy: 0.7014 - loss: 0.5536 - val_accuracy: 0.6405 - val_loss: 0.6322
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 41s 346ms/step - accuracy: 0.6998 - loss: 0

In [44]:
#bilstm
# Step 1: Evaluate the model on the dev set (already done above)
val_loss, val_accuracy = model_bilstm.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Step 2: Generate predictions on the dev set
y_dev_pred = model_bilstm.predict(X_dev)

# Convert the predictions to binary labels (threshold = 0.5)
y_dev_labels = (y_dev_pred > 0.5).astype(int)

# Print the classification report for the dev set
from sklearn.metrics import classification_report
print("Classification Report on Dev Set:")
print(classification_report(y_dev, y_dev_labels))

# Step 3: Predict on the test set
# Load the test data (make sure you use the same tokenizer and padding as training)
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')

# Assuming the test data has a 'text' column
test_texts = test_data['text'].values

# Tokenize and pad the test texts (ensure you use the same tokenizer as training)
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Predict the labels for the test set
test_predictions = model_bilstm.predict(test_padded)

# Convert predictions to binary labels (threshold = 0.5)
test_labels = (test_predictions > 0.5).astype(int).flatten()

# Add the predicted labels to the test dataframe
test_data['predicted_label'] = test_labels

# Step 4: Save the test predictions to a CSV file
output_path = '/content/bilstm_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")

# Step 5: Clean the predictions (mapping numeric values to human-readable labels)
label_mapping = {0: 'Fake', 1: 'original'}
test_data['predicted_label'] = test_data['predicted_label'].map(label_mapping)

# Keep only the 'Id' and 'predicted_label' columns for final submission
output_data = test_data[['Id', 'predicted_label']]

# Step 6: Save the cleaned predictions to a new CSV file
final_output_path = '/content/bilstm_cleaned_test.csv'
output_data.to_csv(final_output_path, index=False)

print(f"Cleaned predictions saved to {final_output_path}")

# Step 7: Evaluate the predictions with the actual labels (if available)
# Load the actual test labels (assuming they are available in another file)
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')

# Add the 'Id' column to match with the predictions file
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Merge predictions and actual labels on 'Id'
merged = pd.merge(output_data, actual_file, on='Id')

# Extract the true and predicted labels for evaluation
y_true = merged['label']  # Actual labels
y_pred = merged['predicted_label']  # Predicted labels

# Compare predictions and compute metrics
from sklearn.metrics import accuracy_score, classification_report

# Accuracy
print("Accuracy:", accuracy_score(y_true, y_pred))

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Optional: Save the merged file with comparison (if needed)
comparison_output_path = '/content/bilstm_merged_comparison.csv'
merged.to_csv(comparison_output_path, index=False)
print(f"Merged comparison file saved to {comparison_output_path}")


26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 202ms/step - accuracy: 0.6511 - loss: 0.6376
Validation Loss: 0.6260471343994141, Validation Accuracy: 0.6527607440948486
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step
Classification Report on Dev Set:
              precision    recall  f1-score   support

           0       0.80      0.41      0.54       406
           1       0.60      0.90      0.72       409

    accuracy                           0.65       815
   macro avg       0.70      0.65      0.63       815
weighted avg       0.70      0.65      0.63       815

32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step
Predictions saved to /content/bilstm_predicted_test.csv
Cleaned predictions saved to /content/bilstm_cleaned_test.csv
Accuracy: 0.6526005888125613
Classification Report:
              precision    recall  f1-score   support

        Fake       0.82      0.39      0.53       507
    original       0.60      0.91      0.73       512

    accuracy                           0.65      1019
   macro avg      